# Importing libraries

In [15]:
pip install nltk

In [22]:
import numpy as np
import random
import json
import pickle
#Deep learning
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
#from keras.optimizers import SGD
#NLP
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

# Initialisation of the data

In [2]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving intents.json to intents.json
User uploaded file "intents.json" with length 3844 bytes


In [8]:
intents_file=open('intents.json').read()
intents= json.loads(intents_file)

In [10]:
intents

{'intents': [{'context': [''],
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'context': [''],
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'tag': 'goodbye'},
  {'context': [''],
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'tag': 'thanks'},
  {'context': [''],
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'tag': 'noanswer'},
  {'context': [''],
   'patterns': ['How you could help me?',
    'What you can do?

# Preprocessing

Simplify the raw data

In [17]:
words=[]
classes=[]
documents=[]
punctuation=['.','!',',','?']
for intent in intents['intents']: #just in case we have a lot of intents but in this cas we only have one intent
  for pattern in intent['patterns']:
    #tokenizing words
    word=nltk.word_tokenize(pattern)
    words.extend(word)
    #add the doc
    documents.append((word,intent['tag']))
    #add classes
    if intent['tag'] not in classes:
      classes.append(intent['tag'])


In [18]:
print(documents)

[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?'], 'adverse_drug'), (['Open', 'adverse', 'drugs', 'module'], 'adverse_drug'), (['Give', 'm

In [19]:
print(classes)

['greeting', 'goodbye', 'thanks', 'options', 'adverse_drug', 'blood_pressure', 'blood_pressure_search', 'pharmacy_search', 'hospital_search']


Lemmatize the words and remove duplicates

In [23]:
lemmatizer=WordNetLemmatizer()
words=[lemmatizer.lemmatize(word.lower()) for word in words if word not in punctuation]
words=sorted(list(set(words)))
# sort classes
classes=sorted(list(set(classes)))

In [26]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

#Create training data

In [27]:
training=[]
output_empty=np.zeros(len(classes))
for doc in documents:
  bag=[]
  word_patterns = doc[0]
  word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)
  output_row=list(output_empty)
  output_row[classes.index(doc[1])]=1
  training.append([bag,output_row])
random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])

Training data created


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  del sys.path[0]


# Create the neural network

In [28]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [30]:
sgd = keras.optimizers.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [31]:
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [32]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

Epoch 1/200
10/10 [==============================] - 1s 5ms/step - loss: 2.2716 - accuracy: 0.0426 
Epoch 2/200
10/10 [==============================] - 0s 3ms/step - loss: 2.1277 - accuracy: 0.1915
Epoch 3/200
10/10 [==============================] - 0s 3ms/step - loss: 2.0798 - accuracy: 0.1915
Epoch 4/200
10/10 [==============================] - 0s 3ms/step - loss: 1.9801 - accuracy: 0.3404
Epoch 5/200
10/10 [==============================] - 0s 3ms/step - loss: 1.9230 - accuracy: 0.4255
Epoch 6/200
10/10 [==============================] - 0s 5ms/step - loss: 1.8405 - accuracy: 0.4468
Epoch 7/200
10/10 [==============================] - 0s 3ms/step - loss: 1.6670 - accuracy: 0.3830
Epoch 8/200
10/10 [==============================] - 0s 3ms/step - loss: 1.4413 - accuracy: 0.5532
Epoch 9/200
10/10 [==============================] - 0s 3ms/step - loss: 1.3583 - accuracy: 0.7021
Epoch 10/200
10/10 [==============================] - 0s 3ms/step - loss: 1.1453 - accuracy: 0.7660
Epoch 11